In [2]:
import os
import glob
import sys
sys.path.append('/Users/aghavamp/Desktop/Projects')
sys.path.append('/Users/aghavamp/Desktop/Projects/bimanual_wrist')
sys.path.append('/Users/aghavamp/Desktop/Projects/Functional_Fusion')
# sys.path.append('/Users/aghavamp/Desktop/PcmPy')
import getpass
import importlib
from tqdm import tqdm
from pathlib import Path

import rsatoolbox as rsa
from rsatoolbox.io import spm as spm_io
import nibabel as nb
import nitools as nt
# import PcmPy as pcm
from PcmPy import PcmPy as pcm
import Functional_Fusion.atlas_map as am
import Functional_Fusion.reliability as rel

import matplotlib.pyplot as plt
import matplotlib
from matplotlib.cm import ScalarMappable
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import matplotlib.patches as patches
import seaborn as sns

import scipy.io as sio
import numpy as np
import pandas as pd

import utils
importlib.reload(utils)

# SET PATHS:
baseDir = os.path.join('/Users', getpass.getuser(), 'Desktop', 'Projects', 'bimanual_wrist', 'data', 'fMRI')
bidsDir = 'BIDS'
anatomicalDir = 'anatomicals'
freesurferDir = 'surfaceFreesurfer'
surfacewbDir = 'surfaceWB' 
behavDir = 'behavioural'
regDir = 'ROI'
atlasDir = '/Volumes/diedrichsen_data$/data/Atlas_templates/fs_LR_32'
analysisDir = os.path.join(os.path.dirname(os.path.dirname(baseDir)), 'analysis')




ModuleNotFoundError: No module named 'PcmPy.matrix'

In [2]:
rois = [
    'M1_L', 'M1_R', 'S1_L', 'S1_R', 'PMd_L', 'PMd_R', 
    'PMv_L', 'PMv_R', 'SMA_L', 'SMA_R', 'V1_L', 'V1_R', 
    'SPLa_L', 'SPLa_R', 'SPLp_L', 'SPLp_R'
]
conditions = ['lhand', 'rhand']
labels = ['flx', 'flxup', 'flxdn', 'ext', 'extup', 'extdn']
cmap = 'RdBu_r' 
sn_bad = []


rois = ['M1_L']
conditions = ['rhand']

# Null model:
ncond = 6
M1 = pcm.model.FixedModel('null', np.eye(ncond))

# Cosine tuning model:
theta = np.array([0, 60, 60, 180, 120, 120])
delta_theta = np.repeat(theta, ncond) - np.tile(theta, ncond)
covs = np.cos(np.deg2rad(delta_theta))
# turn covs into a matrix:
M2 = pcm.model.FixedModel('cosine tuning', covs.reshape((ncond, ncond)))

# noise ceiling model:
M3 = pcm.model.FreeModel('ceil',ncond)

M = [M1, M2, M3]

fix, ax = plt.subplots(1,2, figsize=(8,4))
for i in range(len(M)-1):
    ax[i].imshow(M[i].G, cmap=cmap, vmin=-1, vmax=1)
    ax[i].set_title(M[i].name)
    plt.colorbar(ax[i].imshow(M[i].G, cmap=cmap, vmin=-1, vmax=1), ax=ax[i])
plt.show()

# --- Main Plotting Loop ---
for condition in conditions:
    for i, roi in enumerate(rois):
        # --- Data Loading ---
        file_path = os.path.join(analysisDir, f'pwbeta_{roi}.npz')
        loaded = np.load(file_path, allow_pickle=True)

        # --- Data Preprocessing ---
        subject = loaded['subject']
        indices = [k for k, s in enumerate(subject) if s not in sn_bad]
        
        subject = loaded['subject'][indices]
        cond_name_tmp = loaded['cond_intrinsic_vec'][indices]
        cond_vec_tmp = loaded['cond_num_vec'][indices]
        data_tmp = loaded['beta_white'][indices]
        part_tmp = loaded['part_vec'][indices]

        data, cond_vec, part_vec = [], [], []
        
        for i_sub in range(len(subject)):
            # Filter for the current condition
            idx = [j for j, c in enumerate(cond_name_tmp[i_sub]) if condition in c]
            data.append(data_tmp[i_sub][idx, :])
            cond_vec.append(cond_vec_tmp[i_sub][idx])
            part_vec.append(part_tmp[i_sub][idx])

        Y = []
        for i_sub in range(len(data)):
            obs_des = {'cond_vec': cond_vec[i_sub], 'part_vec': part_vec[i_sub]}
            Y.append(pcm.dataset.Dataset(data[i_sub], obs_descriptors=obs_des))
        
        T_in, theta_in = pcm.fit_model_individ(Y,M,fit_scale=True, fixed_effect='block', verbose=True)
#         N = len(Y)
#         ncond = len(np.unique(Y[0].obs_descriptors['cond_vec']))
        
#         G_hat = np.zeros((N, ncond, ncond))
#         for i_sub in range(N):
#             G_hat[i_sub, :, :], _ = pcm.est_G_crossval(Y[i_sub].measurements,
#                                                         Y[i_sub].obs_descriptors['cond_vec'],
#                                                         Y[i_sub].obs_descriptors['part_vec'])
        
#         G_hat_avg = np.mean(G_hat, axis=0)
#         D_avg = pcm.G_to_dist(G_hat_avg)
#         D_avg = np.sign(D_avg) * np.sqrt(np.abs(D_avg))
        

T_in



AttributeError: module 'PcmPy' has no attribute 'model'